# Project 02 - No.8
## 網路股票討論度與波動度關係之探討

In [32]:
import requests
import time
from bs4 import BeautifulSoup
import os
import re
import urllib.request
import json
import jieba
import jieba.analyse

In [33]:
PTT_URL = 'https://www.ptt.cc'

In [68]:
def get_web_page(url):
    time.sleep(0.5)  # 每次爬取前暫停 0.5 秒以免被 PTT 網站判定為大量惡意爬取
    resp = requests.get(url=url)
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [94]:
def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html.parser')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').string.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            if d.find('div', 'nrec').string:
                try:
                    push_count = int(d.find('div', 'nrec').string)  # 轉換字串為數字
                except ValueError:  # 若轉換失敗，不做任何事，push_count 保持為 0
                    pass

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').string
                articles.append(
                    title,
                )
    return articles, prev_url


In [95]:
def parse(dom):
    soup = BeautifulSoup(dom, 'html.parser')
    links = soup.find(id='main-content').find_all('a')
    img_urls = []
    for link in links:
        if re.match(r'^https?://(i.)?(m.)?imgur.com', link['href']):
            img_urls.append(link['href'])
    return img_urls


In [96]:
def save(img_urls, title):
    if img_urls:
        try:
            dname = title.strip()  # 用 strip() 去除字串前後的空白
            os.makedirs(dname)
            for img_url in img_urls:
                if img_url.split('//')[1].startswith('m.'):
                    img_url = img_url.replace('//m.', '//i.')
                if not img_url.split('//')[1].startswith('i.'):
                    img_url = img_url.split('//')[0] + '//i.' + img_url.split('//')[1]
                if not img_url.endswith('.jpg'):
                    img_url += '.jpg'
                fname = img_url.split('/')[-1]
                urllib.request.urlretrieve(img_url, os.path.join(dname, fname))
        except Exception as e:
            print(e)

In [102]:
if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Stock/index.html')
    if current_page:
        articles = []  # 全部的今日文章
        date = time.strftime("01/02").lstrip('0') # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url = get_articles(current_page, date)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, date)
            
        # 已取得文章列表，開始進入各文章讀圖
        
        for article in articles:
            print(article)
            

In [103]:
print (articles)

[]


In [99]:
keywords=jieba.analyse.extract_tags(str(articles),topK=100)

In [100]:
print("/".join(keywords))

新聞/請益/標的/違反/板規/強勢/水桶/Re/蘋概/GIS/美律/權證/N5D6M6C/10/efun0242/一週/近幾年/積電/一樣/銀行/保險業/動作/判斷/匯率/走勢/友訊/2332/帶動/股價/漲停/type/台灣/有肉/世紀/9958/2395/研華/買到/樂透/什麼/財經/觀測站/著大/股東/進出/沒錯/進口車/進逼/u3000/執行長/凱泰/6462/神盾/16/大戶/領攻/11/000/宏達/12/月營/收僅/40/億元/創近/13/年次/公告/裕隆/利多/下周/布局/心情/真的/股票/可以
